<a href="https://colab.research.google.com/github/cyyeh/time-series-playground/blob/main/pytorch-101/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart

reference: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## [Working with data](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#working-with-data)

1. PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the Dataset.

2. `DataLoader` represents a Python iterable over a dataset, with support for
- [map-style and iterable-style datasets](https://pytorch.org/docs/stable/data.html#dataset-types)
- [customizing data loading order](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler)
- [automatic batching](https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data)
- [single- and multi-process data loading](https://pytorch.org/docs/stable/data.html#single-and-multi-process-data-loading)
- [automatic memory pinning](https://pytorch.org/docs/stable/data.html#memory-pinning)

3. PyTorch offers domain-specific libraries such as `TorchText`, `TorchVision`, and `TorchAudio`, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f'Shape of X[N, C, H, W]: {X.shape}')
  print(f'Shape of y: {y.shape} {y.dtype}')
  break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## [Creating Models](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models)

To define a neural network in PyTorch, we create a class that inherits from `nn.Module`. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU if available.

In [6]:
# Get cpu or gpu device for training.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

# Define model.
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## [Optimizing the Model Parameters](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#optimizing-the-model-parameters)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)
  
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= size
  correct /= size
  print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')


epochs = 5
for t in range(epochs):
  print(f'Epoch {t+1}\n----------------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model)
print('Done')

Epoch 1
----------------------------
loss: 2.306327 [    0/60000]
loss: 2.298875 [ 6400/60000]
loss: 2.288821 [12800/60000]
loss: 2.281961 [19200/60000]
loss: 2.280726 [25600/60000]
loss: 2.245951 [32000/60000]
loss: 2.257060 [38400/60000]
loss: 2.230825 [44800/60000]
loss: 2.234756 [51200/60000]
loss: 2.200581 [57600/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 0.034874 

Epoch 2
----------------------------
loss: 2.232012 [    0/60000]
loss: 2.240308 [ 6400/60000]
loss: 2.196250 [12800/60000]
loss: 2.199338 [19200/60000]
loss: 2.193954 [25600/60000]
loss: 2.112939 [32000/60000]
loss: 2.153031 [38400/60000]
loss: 2.094304 [44800/60000]
loss: 2.120254 [51200/60000]
loss: 2.043931 [57600/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 0.032888 

Epoch 3
----------------------------
loss: 2.120716 [    0/60000]
loss: 2.134830 [ 6400/60000]
loss: 2.043498 [12800/60000]
loss: 2.064970 [19200/60000]
loss: 2.022579 [25600/60000]
loss: 1.890764 [32000/60000]
loss: 1.963221 [38400/60000]
lo

## [Saving Models](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#saving-models)




In [9]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


## [Loading Models](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#loading-models)

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [13]:
classes = [
  "T-shirt/top",
  "Trouser",
  "Pullover",
  "Dress",
  "Coat",
  "Sandal",
  "Shirt",
  "Sneaker",
  "Bag",
  "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: {predicted}, Actual: {actual}')

Predicted: Ankle boot, Actual: Ankle boot
